<a href="https://colab.research.google.com/github/falahgithub/public_repos/blob/main/Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# * Steps that are followed in following program *

### Step 0 : Saving CSV files into the database
### Step 1 : Importing files from the database and converting them into dataframes
### Step 2 : Checking whether all the dataframes contains required columns
### Step 3 : Analyzing dataframes along with Data Cleaning and Manipulation
### Step 4 : Calculating threshold to filter rows of dataframe depending upon situations
### Step 5 : Data calculations to generate results

# Code to save data in the database

In [ ]:
def files_to_database():
  # Read CSV data into a DataFrame

  df1_export = pd.read_csv('store status.csv')
  df2_export = pd.read_csv('Menu hours.csv')
  df3_export = pd.read_csv('bq-results-20230125-202210-1674678181880.csv')

  # Connect to SQLite database

  conn = sqlite3.connect('mydb.db')

  # Insert data into a table

  df1_export.to_sql('T1', conn, if_exists='replace')
  df2_export.to_sql('T2', conn, if_exists='replace')
  df3_export.to_sql('T3', conn, if_exists='replace')

  # Close the database connection

  conn.close()

# Code to import tables from database

In [ ]:
def importing_tables():

  """ This function imports CSV files from the database and converts them into the dataframes."""

  # Connect to the database

  conn = sqlite3.connect('mydb.db')
  cursor = conn.cursor()

  # Export the first table

  cursor.execute('SELECT * FROM T1')
  rows = cursor.fetchall()
  with open('table1.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow([description[0] for description in cursor.description])
      writer.writerows(rows)

  # Export the second table

  cursor.execute('SELECT * FROM T2')
  rows = cursor.fetchall()
  with open('table2.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow([description[0] for description in cursor.description])
      writer.writerows(rows)

  # Export the third table

  cursor.execute('SELECT * FROM T3')
  rows = cursor.fetchall()
  with open('table3.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      writer.writerow([description[0] for description in cursor.description])
      writer.writerows(rows)

  # Close the database connection

  conn.close()

  # Converting CSV to Dataframes

  df1 = pd.read_csv("table1.csv").drop("index", axis=1)
  df2 = pd.read_csv("table2.csv").drop("index", axis=1)
  df3 = pd.read_csv("table3.csv").drop("index", axis=1)

# Checking whether all the dataframes contains required columns

In [ ]:
def checking_imports(*dfs: pd.DataFrame): 
  
  # Validate input data for each DataFrames

  for df in dfs:
    if not isinstance(df, pd.DataFrame):
      raise TypeError("All dataframes must be pandas DataFrames.")

    if not {'store_id'}.issubset(df.columns):
       raise ValueError("All dataframes must contain 'store_id' column.")

# Analyzing dataframes along with Data Cleaning and Manipulation

In [ ]:
def analyz_clean_tables(df1, df2, df3):
  
  """This function analyzes all the dataframes, perform data cleaning function and finally manipulate dataframes as required by the problem. It takes dataframes as input.""" 
  
  # Data information

  df1.info(), df2.info(), df3.info()

  # Merge dataframe on "store_id"

  df13 = pd.merge(df1, df3, how="outer", on="store_id")

  # Data Cleaning

  df13.fillna({"timezone_str": "America/Chicago"}, inplace=True)
  df13 = df13.dropna()

  # Data manipulation

  df13["timestamp_utc"] = df13["timestamp_utc"].apply(lambda x: x.replace(" UTC", ""))
  df13["timestamp_utc"] = pd.to_datetime(df13["timestamp_utc"], utc = True)
  df13["timestamp_local"] = df13.apply(lambda row: row["timestamp_utc"].tz_convert(row["timezone_str"]), axis=1)          # remember axisdf13
  df13["day"] = df13["timestamp_local"].apply(lambda x: x.dayofweek)       

  # Merge all dataframes on "store_id"

  df123 = pd.merge(df13, df2,how="left" , on=["store_id", "day"])   

  # Data Cleaning

  df123 = df123.fillna({"start_time_local": "00:00:00","end_time_local": "23:59:59"})
  df123["end_time_local"].replace(to_replace="00:00:00", value="23:59:59", inplace=True)  
  
  return df123
  

# Calculating threshold to filter rows of dataframe depending upon situations

In [ ]:
def threshold_calc(df):
  
  """This function is used to calculate threshold to filter rows of dataframe depending upon situations. Here 1 hr, 1 day, and 1 week."""

  # Thresholds calculation to get desired rows

  end_th = df['timestamp_utc'].max()
  hour_th = df['timestamp_utc'].max() - pd.Timedelta(hours=1)
  day_th = df['timestamp_utc'].max() - pd.Timedelta(days=1)
  week_th = df['timestamp_utc'].max() - pd.Timedelta(weeks=1)

  return end_th, hour_th, day_th, week_th          

# Data calculations to generate results

In [ ]:
def proces_group_data(data, end_th, hour_th, day_th, week_th):
  
  """This function filters rows of dataframe depending upon the threshold calculated. It returns uptime/downtime calculated."""
    
  last_hour = data[data['timestamp_utc'] > hour_th]        # filtering rows within an hour
  last_day = data[data['timestamp_utc'] > day_th]          # filtering rows within a day
  last_week = data[data['timestamp_utc'] > week_th]        # filtering rows within a week

  u_h, d_h = calvall(last_hour, hour_th, end_th)  
  u_d, d_d = calvall(last_day, day_th, end_th)     
  u_w, d_w = calvall(last_week, week_th, end_th)  

  return u_h, u_d, u_w, d_h, d_d, d_w  

In [ ]:
def calvall(data, start_threshold, end_threshold):

  """This function calculates uptime/downtime. It returns uptime and downtime values."""
  
  uptime = 0
  downtime = 0
 
  if data.empty:
    pass
  
  else:
    newgroups = data.groupby("day")                                                        # Grouping dataframe on different days

    for gr_name, gr_data in newgroups: 
      if gr_data.empty:
        pass
      else:
        
        timezone = gr_data["timezone_str"].iloc[0]                                          # timezone identification
        
        
        start_cutoff = start_threshold.tz_convert(timezone)                                 # For cutoff limit
        end_cutoff = end_threshold.tz_convert(timezone)                                     # For cutoff limit
      
        
        s_time = pd.to_datetime(gr_data["start_time_local"].iloc[0]).time()                  # Business hours start time
        e_time = pd.to_datetime(gr_data["end_time_local"].iloc[0]).time()                    # Business hours end time
        starting_hr = int(s_time.strftime("%H"))                                             # Extracting hours from start time
        ending_hr = int(e_time.strftime("%H"))                                               # Extracting hours from end time

        
        gr_data = gr_data.sort_values("timestamp_local")                                     # sorting dataframe
        gr_data["status_code"] = gr_data["status"].replace({"active": 1, "inactive": 0})
        gr_data = gr_data[["timestamp_local", "status_code"]]    
        gr_data = gr_data.set_index("timestamp_local")                                        
        gr_data = gr_data.resample("1H").max()                                                # Resampling data
        
      
       
        start_pos = gr_data.index[0].replace(hour=starting_hr, minute = 0, second = 0)         # Business start time with current date
        end_pos = gr_data.index[0].replace(hour=ending_hr, minute = 0, second = 0)             # Business end time with current date


        gr_data = gr_data.reindex(pd.date_range(start_pos, end_pos, freq='1H'))                # Reindexing
        
        
        try:
          gr_data = gr_data.interpolate(method= "time", limit_direction="both")
        
        except:
          gr_data = gr_data.fillna({"status_code":1})
        
        
        gr_data = gr_data.query('@start_cutoff <= index <= @end_cutoff')                       # Filtering dataframe within cutoff limits
                  
        for index, row in gr_data.iterrows():
          if row["status_code"] > 0.66 :
             uptime += 1
          else:
            downtime += 1   
  
  return uptime, downtime

In [ ]:
# Create empty lists to store value for output schema
def main():

  """This function is responsible for coordinating between different functions."""
  
  files_to_database()
  df1, df2, df3 = importing_tables()                                      # Step 1
  checking_imports(df1, df2, df3)                                         # Step 2
  df = analyz_clean_tables(df1, df2, df3)                                 # Step 3
  end_th, hour_th, day_th, week_th = threshold_calc(df)                   # Step 4


  store = []
  u_h_list = []
  u_d_list = [] 
  u_w_list = []
  d_h_list = []
  d_d_list = [] 
  d_w_list = []

  # Create groups object

  groups = df.groupby("store_id")

  for name, data in groups:
    store.append(name)  
    u_h, u_d, u_w, d_h, d_d, d_w  = proces_group_data(data, end_th, hour_th, day_th, week_th)
    u_h_list.append(u_h*60)                # 60 as output is required in minutes
    u_d_list.append(u_d) 
    u_w_list.append(u_w)
    d_h_list.append(d_h*60)                # 60 as output is required in minutes
    d_d_list.append(d_d) 
    d_w_list.append(d_w) 


  # Required output dataframe

  dfgen = pd.DataFrame({"store_id": store,
                      "uptime_last_hour(in minutes)":u_h_list,
                      "uptime_last_day(in hours)": u_d_list,
                      "uptime_last_week(in hours)" : u_w_list,
                      "downtime_last_hour(in minutes)": d_h_list,
                      "downtime_last_day(in hours)": d_d_list,
                      "downtime_last_week(in hours)": d_w_list,
                      })
  
  dfgen.to_csv("result.csv", index=False)

In [ ]:
import pandas
import sqlite3
import csv

main()

In [ ]:
res = pd.read_csv("result.csv")
res